# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../.."))

In [2]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from utilities.losses import MseCrossEntropyCombination
from elements.abstracts import AbstractModulator
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from elements.simple import AdjustSize, AmplitudeToIntensity, IntensityToAmplituder, Empty
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
import torch
import torchvision
import numpy
import timm
import pandas
import json
from copy import deepcopy
from cluster import train, confusion, execute, epochs, SelectedGPUs
from PIL import Image
from itertools import product
import optuna

In [3]:
FontLibrary.Fonts.PlotTitle.FontSize = 16

FontLibrary.Fonts.DescriptionLeft.FontSize = 12
FontLibrary.Fonts.DescriptionLeft.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionLeft.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionBottom.FontSize = 10
FontLibrary.Fonts.DescriptionBottom.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionBottom.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionTop.FontSize = 10
FontLibrary.Fonts.DescriptionTop.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionTop.FontStyle = 'italic'

FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FontLibrary.Fonts.AxisX.FontSize = 8
FontLibrary.Fonts.AxisY.FontSize = 8

FontLibrary.Fonts.GraphTitle.FontSize = 10

FigureWidthHeight = (6.69291, 10.1181-1.0)

In [4]:
device = torch.device(2 if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA A100-SXM4-80GB


# Определение методов

In [5]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=False, in_chans=1, num_classes=classes).to(device)

# Определение цели для Optuna

In [6]:
def objective(trial:optuna.trial):
    # Постоянные параметры 
    N = 512 #1536
    pixels = 512 #1536
    length = 3.0E-3
    wavelength = 500.0E-9
    spatial_coherence = 100.0E-6
    time_coherence = 10.0E-9
    time_relaxation = 1.0E-6
    mean_samples = 20 #20
    detectors_amount = 24
    detector_size = length / 60 

    # Записываем постоянные параметры
    trial.set_user_attr('N',                 N)
    trial.set_user_attr('pixels',            pixels)
    trial.set_user_attr('length',            length)
    trial.set_user_attr('wavelength',        wavelength)
    trial.set_user_attr('spatial_coherence', spatial_coherence)
    trial.set_user_attr('time_coherence',    time_coherence)
    trial.set_user_attr('time_relaxation',   time_relaxation)
    trial.set_user_attr('mean_samples',      mean_samples)
    trial.set_user_attr('detectors',         detectors_amount)
    trial.set_user_attr('detectors_size',    detector_size)
    
    # Варьируемые параметры
    masks_amount              = trial.suggest_int('masks amount',               3,      10)
    batch_size                = trial.suggest_int('batch size',                 32//8,  40//8)
    distance                  = trial.suggest_float('distance',                 5.0E-3, 500.0E-3)
    learning_rate             = trial.suggest_float('learning rate',            1.0E-6, 1.0E-2)
    loss_functions_proportion = trial.suggest_float('cross entorpy proportion', 0.0,    1.0)
    optimizer_type            = trial.suggest_categorical('optimizer type',     ["Adam", "RMSprop"])
    optimizer_type = {"Adam":torch.optim.Adam, "RMSprop":torch.optim.RMSprop}[optimizer_type]
    loss_function = MseCrossEntropyCombination(loss_functions_proportion)
    
    # Определение датасета
    dataset = Dataset('CIFAR10', batch_size, N, N, torch.float32, threads=8, preload=10)
    # dataset.padding(surface_ratio=0.3)
    dataset.load.train()
    dataset.load.test()

    # Модуль некогерентности
    incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, N, length)
    incoherent_encoder, incoherent_decoder = incoherent.pair()
    incoherent_encoder, incoherent_decoder = incoherent_encoder.to(device), incoherent_decoder.to(device)
    incoherent.sample()
    incoherent_encoder, incoherent_decoder = incoherent_encoder.cpu(), incoherent_decoder.cpu()
    torch.cuda.empty_cache()

    # Инициализация детекторов
    spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
    detectors_filter = Gaussian((detector_size, detector_size), (0,0))
    detectors = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter)

    # Инициализация дополнительных можулей
    int_to_amp = IntensityToAmplituder()
    full_propagation =  FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance*masks_amount, 0.4)
    
    # Инициализация электронной, оптической части и гибридной модели
    electronic = timm.create_model('resnet18', pretrained=False,  in_chans=1, num_classes=10)
    propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
    phase_modulators     = [PhaseModulator(N, length, pixels)     for i in range(masks_amount)]
    amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
    elements = [phase_modulators[0], amplitude_modulators[0]]
    for phase_modulator, amplitude_modulator in zip(phase_modulators[1:], amplitude_modulators[1:]):
        elements.append(propagation)
        elements.append(phase_modulator)
        elements.append(amplitude_modulator)
    elements.append(propagation)
    optical = CompositeModel(int_to_amp, incoherent_encoder, *elements, incoherent_decoder)
    model = HybridModel(optical, detectors, electronic)

    # Инициализация пустой модели (без оптической части)
    empty_incoherent = deepcopy(incoherent)
    empty_incoherent_encoder, empty_incoherent_decoder = empty_incoherent.pair()
    empty_optical = CompositeModel(int_to_amp, empty_incoherent_encoder, full_propagation, empty_incoherent_decoder)
    empty_model = HybridModel(empty_optical, deepcopy(detectors), deepcopy(electronic))

    # Обучение моделей
    mh, lh, cmh, emh, elh, ecmh = [], [], [], [], [], []
    torch.cuda.empty_cache()
    mh, lh, cmh    = epochs(2, 10, model,       dataset, loss_function, optimizer_type, lr=learning_rate)
    torch.cuda.empty_cache()
    emh, elh, ecmh = epochs(2, 10, empty_model, dataset, loss_function, optimizer_type, lr=learning_rate)

    # Запись результатов
    accuracy = 100*numpy.sum(numpy.diagonal(cmh[-1], 0))/numpy.sum(cmh[-1])
    empty_accuracy = 100*numpy.sum(numpy.diagonal(ecmh[-1], 0))/numpy.sum(ecmh[-1])
    trial.set_user_attr('accuracy',       accuracy)
    trial.set_user_attr('empty_accuracy', empty_accuracy)
    print(f"Точности модели и пустой модели: {accuracy} | {empty_accuracy}")

    # Запись истории ошибок
    lh = numpy.concatenate(lh).tolist()
    trial.set_user_attr('loss_history', json.dumps(lh))
    
    return accuracy

# optuna.delete_study(study_name="ID2NN 2Epochs SR", storage="sqlite:///ID2NN.db")
study = optuna.create_study(study_name="ID2NN_2Epochs SR", storage="sqlite:///ID2NN.db", direction='maximize', load_if_exists=True)

[I 2024-09-10 13:39:58,868] Using an existing study with name 'ID2NN_2Epochs SR' instead of creating a new one.


In [15]:
dataframe:pandas.DataFrame = study.trials_dataframe()
dataframe.nlargest(30, "user_attrs_accuracy")[["user_attrs_empty_accuracy", "user_attrs_accuracy"]].sort_values("user_attrs_accuracy")

,user_attrs_empty_accuracy,user_attrs_accuracy
26,20.076000,19.338000
27,27.162000,20.076000
20,23.847631,20.849023
64,36.322000,21.538892
65,48.349472,23.375081
47,28.894829,24.136000
33,29.214000,24.242880
49,26.380658,24.596000
25,19.338000,26.112000
13,51.644000,26.224145


# Оптимизация

In [ ]:
study.optimize(objective, n_trials=30)

Training main thread PID is: 2279821
| 1724403137.9229205 : 4 : Start Training
| 1724403137.92545 : 1 : Start Training
| 1724403137.93011 : 0 : Start Training
| 1724403137.9313984 : 6 : Start Training
| 1724403137.9325755 : 3 : Start Training
| 1724403137.9331515 : 7 : Start Training
| 1724403137.9335358 : 5 : Start Training
| 1724403137.9337003 : 2 : Start Training
RL:1.929, RPI1K:-140.21·10⁻³: 100%|███████████████ | 1250/1250 | 07:07m - 00:00m| 1724403567.4485588 : 2 : Training finished, gathering
| 1724403567.4492834 : 2 : Gathering finished, start validation
| 1724403567.5472074 : 4 : Training finished, gathering
| 1724403567.547864 : 4 : Gathering finished, start validation
| 1724403567.6237783 : 7 : Training finished, gathering
| 1724403567.62424 : 7 : Gathering finished, start validation
| 1724403567.6683679 : 5 : Training finished, gathering
| 1724403567.6688852 : 5 : Gathering finished, start validation
| 1724403567.683981 : 3 : Training finished, gathering
| 1724403567.684446

[I 2024-08-23 09:39:17,877] Trial 43 finished with value: 33.38900975206195 and parameters: {'masks amount': 8, 'batch size': 5, 'distance': 0.2785769272992344, 'learning rate': 0.007125931432594686, 'cross entorpy proportion': 0.9562681740803972, 'optimizer type': 'Adam'}. Best is trial 14 with value: 51.644.


Точности модели и пустой модели: 33.38900975206195 | 28.264
Training main thread PID is: 2354200
| 1724405998.2514327 : 0 : Start Training
| 1724405998.2543805 : 4 : Start Training
| 1724405998.2557216 : 6 : Start Training
| 1724405998.2560568 : 5 : Start Training
| 1724405998.2578022 : 3 : Start Training
| 1724405998.2586153 : 2 : Start Training
| 1724405998.2617157 : 1 : Start Training
| 1724405998.2627606 : 7 : Start Training
RL:1.502, RPI1K:-103.78·10⁻³: 100%|███████████████ | 1562/1562 | 06:22m - 00:00m| 1724406382.8088348 : 2 : Training finished, gathering
| 1724406382.8093245 : 2 : Gathering finished, start validation
| 1724406382.8182356 : 6 : Training finished, gathering
| 1724406382.818768 : 6 : Gathering finished, start validation
| 1724406382.8263154 : 5 : Training finished, gathering
| 1724406382.8267314 : 5 : Gathering finished, start validation
| 1724406382.8474333 : 4 : Training finished, gathering
| 1724406382.8479238 : 4 : Gathering finished, start validation
| 172440

[I 2024-08-23 10:04:53,518] Trial 44 finished with value: 10.140830647150995 and parameters: {'masks amount': 7, 'batch size': 4, 'distance': 0.17040058027183325, 'learning rate': 0.0016128066764965975, 'cross entorpy proportion': 0.7311833059712475, 'optimizer type': 'RMSprop'}. Best is trial 14 with value: 51.644.


Точности модели и пустой модели: 10.140830647150995 | 28.34032723258795
Training main thread PID is: 2427159
| 1724407534.081223 : 0 : Start Training
| 1724407534.0816243 : 7 : Start Training
| 1724407534.0833442 : 1 : Start Training
| 1724407534.084084 : 2 : Start Training
| 1724407534.0872366 : 4 : Start Training
| 1724407534.0874524 : 5 : Start Training
| 1724407534.08831 : 6 : Start Training
| 1724407534.091173 : 3 : Start Training
RL:1.11, RPI1K:-22.44·10⁻³: 100%|█████████████████ | 1250/1250 | 04:44m - 00:00m| 1724407820.2382934 : 1 : Training finished, gathering
| 1724407820.2387881 : 1 : Gathering finished, start validation
| 1724407820.2632453 : 3 : Training finished, gathering
| 1724407820.2637432 : 3 : Gathering finished, start validation
RL:1.11, RPI1K:-22.44·10⁻³: 100%|█████████████████ | 1250/1250 | 04:45m - 00:00m
| 1724407820.331243 : 0 : Training finished, gathering
| 1724407820.3317225 : 0 : Gathering finished, start validation
  0%|                                   

...

Точности модели и пустой модели: 10.020375988106283 | 36.244
Training main thread PID is: 2499762
| 1724408772.4579804 : 3 : Start Training
| 1724408772.4623187 : 6 : Start Training
| 1724408772.464705 : 0 : Start Training
| 1724408772.467827 : 1 : Start Training
| 1724408772.4697475 : 2 : Start Training
| 1724408772.4704504 : 7 : Start Training
| 1724408772.4706268 : 4 : Start Training
| 1724408772.4706967 : 5 : Start Training
RL:2.035, RPI1K:-101.16·10⁻³: 100%|███████████████ | 1250/1250 | 07:09m - 00:00m
| 1724409202.711575 : 0 : Training finished, gathering
| 1724409202.7120652 : 0 : Gathering finished, start validation
  0%|                                                  | 0/1250 [00:00<?, ?it/s]| 1724409202.7225995 : 7 : Training finished, gathering
| 1724409202.7231061 : 7 : Gathering finished, start validation
| 1724409202.7542121 : 6 : Training finished, gathering
| 1724409202.754669 : 6 : Gathering finished, start validation
| 1724409202.761446 : 2 : Training finished, gath

[I 2024-08-23 10:53:17,570] Trial 46 finished with value: 7.86873460691541 and parameters: {'masks amount': 8, 'batch size': 5, 'distance': 0.2716894616615297, 'learning rate': 0.007969921455916188, 'cross entorpy proportion': 0.9853163083449367, 'optimizer type': 'Adam'}. Best is trial 14 with value: 51.644.


Точности модели и пустой модели: 7.86873460691541 | 24.136
Training main thread PID is: 2572894
| 1724410437.9642339 : 4 : Start Training
| 1724410437.9682128 : 0 : Start Training
| 1724410437.969305 : 2 : Start Training
| 1724410437.9723556 : 1 : Start Training
| 1724410437.9724178 : 3 : Start Training
| 1724410437.974371 : 5 : Start Training
| 1724410437.9747884 : 6 : Start Training
| 1724410437.9749668 : 7 : Start Training
RL:1.986, RPI1K:-119.66·10⁻³: 100%|███████████████ | 1250/1250 | 07:08m - 00:00m| 1724410867.9999137 : 7 : Training finished, gathering
| 1724410868.0003936 : 7 : Gathering finished, start validation
RL:1.986, RPI1K:-119.66·10⁻³: 100%|███████████████ | 1250/1250 | 07:09m - 00:00m
| 1724410868.0473225 : 0 : Training finished, gathering
| 1724410868.0478349 : 0 : Gathering finished, start validation
  0%|                                                  | 0/1250 [00:00<?, ?it/s]| 1724410868.0564163 : 6 : Training finished, gathering
| 1724410868.0568857 : 6 : Gather

[I 2024-08-23 11:40:59,090] Trial 47 finished with value: 24.136 and parameters: {'masks amount': 8, 'batch size': 5, 'distance': 0.20778993409787722, 'learning rate': 0.0071977115936612875, 'cross entorpy proportion': 0.9230401060729926, 'optimizer type': 'Adam'}. Best is trial 14 with value: 51.644.


Точности модели и пустой модели: 24.136 | 28.8948289955247
Training main thread PID is: 2647275
| 1724413299.3479784 : 0 : Start Training
| 1724413299.3538036 : 5 : Start Training
| 1724413299.360306 : 1 : Start Training
| 1724413299.361056 : 6 : Start Training
| 1724413299.3618898 : 3 : Start Training
| 1724413299.3626728 : 4 : Start Training
| 1724413299.362943 : 7 : Start Training
| 1724413299.3631482 : 2 : Start Training
RL:1.937, RPI1K:-116.24·10⁻³: 100%|███████████████ | 1250/1250 | 06:20m - 00:00m| 1724413681.1880748 : 3 : Training finished, gathering
| 1724413681.1888812 : 3 : Gathering finished, start validation
| 1724413681.2093701 : 1 : Training finished, gathering
| 1724413681.2101264 : 1 : Gathering finished, start validation
| 1724413681.2117393 : 2 : Training finished, gathering
| 1724413681.2122188 : 2 : Gathering finished, start validation
| 1724413681.225128 : 5 : Training finished, gathering
| 1724413681.2255938 : 5 : Gathering finished, start validation
| 1724413681